In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_3c"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()
PATH = 'soft-os-final'
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc_v1 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [4]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (134s),  symKL_DB_eta: 165.242,  symKL_DB_z: 172.782,  loss: -950.023,  ess: 3.812,  kl_eta_ex: 382.324,  kl_eta_in: 150.861,  kl_z_ex: 102.540,  kl_z_in: 20.916
epoch: 1\250 (133s),  symKL_DB_eta: 71.823,  symKL_DB_z: 40.784,  loss: -759.079,  ess: 4.632,  kl_eta_ex: 126.671,  kl_eta_in: 167.326,  kl_z_ex: 19.970,  kl_z_in: 8.742
epoch: 2\250 (142s),  symKL_DB_eta: 63.108,  symKL_DB_z: 40.467,  loss: -731.795,  ess: 4.733,  kl_eta_ex: 112.244,  kl_eta_in: 174.918,  kl_z_ex: 18.880,  kl_z_in: 8.191
epoch: 3\250 (149s),  symKL_DB_eta: 65.749,  symKL_DB_z: 41.139,  loss: -724.342,  ess: 4.822,  kl_eta_ex: 115.259,  kl_eta_in: 186.936,  kl_z_ex: 18.824,  kl_z_in: 7.537
epoch: 4\250 (153s),  symKL_DB_eta: 74.164,  symKL_DB_z: 38.792,  loss: -718.233,  ess: 4.918,  kl_eta_ex: 125.412,  kl_eta_in: 206.567,  kl_z_ex: 18.695,  kl_z_in: 7.044
epoch: 5\250 (136s),  symKL_DB_eta: 88.644,  symKL_DB_z: 39.270,  loss: -716.551,  ess: 4.989,  kl_eta_ex: 142.189,  kl_eta_in: 222.794,  kl_

KeyboardInterrupt: 

In [ ]:
# torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
# torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
# torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)

In [ ]:
indices = torch.arange(NUM_DATASETS)
step = 2
batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
obs = Data[batch_indices]
obs = shuffler(obs).repeat(SAMPLE_SIZE, 1, 1, 1)
if CUDA:
    obs =obs.cuda().to(DEVICE)
    
q_eta, p_eta, q_nu = oneshot_eta(obs, K, D)
obs_mu = q_eta['means'].value

In [ ]:
obs_mu[0,0]

In [ ]:
q_eta['means'].dist.loc[0]